canonical tautomer -> inchi -> key

script to compute de/protonation energy
-> requires computing energy of neutral state

In [16]:
import pandas as pd
import numpy as np
import hashlib
from pprint import pprint
from rdkit import Chem
from rdkit.Chem import AllChem
from metatlas import create_launchpad, make_df_with_smiles_only_from_csv, OBUFFOptimize, Psi4Optimize, OrcaOptimize
from fireworks import Firework, FWorker, Workflow
from fireworks.core.rocket_launcher import rapidfire, launch_rocket
from tqdm import tqdm_notebook as tqdm
import psi4
from mendeleev import element

from multiprocessing import Pool

In [2]:
df = make_df_with_smiles_only_from_csv(csv_file='chebi_and_metacyc_molecules.csv')

In [3]:
""" If you want to draw SVGs """
import pybel
import xml.etree.ElementTree as ET
from IPython.display import SVG, HTML

def draw_svg(pymol):
    """ molclone should be the 'clone' property of a pybel.Molecule"""
    namespace = "http://www.w3.org/2000/svg"
    ET.register_namespace("", namespace)
    obsvg = pymol.clone.write("svg")
    tree = ET.fromstring(obsvg)
    svg = tree.find("{{{ns}}}g/{{{ns}}}svg".format(ns=namespace))
    return SVG(ET.tostring(svg).decode("utf-8"))

""" If you want to draw html renderings"""
import imolecule

def draw_html(pymol):
    return HTML(imolecule.draw(pymol.clone, 
                               format="pybel", 
                               display_html=False))

""" These two imports required for in-notebook RDKit plotting"""
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw.MolDrawing import MolDrawing

In [4]:
dbini = '/home/bkrull/.fireworks/metatlas.ini'
lpad = create_launchpad(dbini)

data = lpad.get_fw_dict_by_id(108317)
pprint(data)

{u'_id': ObjectId('5aa842d385e51e3f77c5988c'),
 u'archived_launches': [{u'_id': ObjectId('5ad834d1f2c314bc208e49d1'),
                         u'action': None,
                         u'fw_id': 108317,
                         u'fworker': {u'category': u'',
                                      u'env': {},
                                      u'name': u'edison',
                                      u'query': u'{}'},
                         u'host': u'nid06104',
                         u'ip': u'10.128.24.9',
                         u'launch_dir': u'/scratch2/scratchdirs/bkrull/metatlas/block_2018-04-19-05-39-13-162599/launcher_2018-04-19-06-17-18-106840',
                         u'launch_id': 108975,
                         u'runtime_secs': None,
                         u'state': u'RUNNING',
                         u'state_history': [{u'checkpoint': {u'_all_mod_spec': [{u'_push': {u'formula': u'C204H301N51O64',
                                                                  

In [26]:
suspect = pymol.atoms[0]
suspect.type

'Du'

In [27]:
smi = df.loc[7, 'original_smiles']
print smi
pymol = pybel.readstring('smi', smi)
pymol.addh()
pymol.make3D()
for atom in pymol.atoms:
    print '{}, {} ({}, {}, {})'.format(
        atom.atomicnum, atom.type,
        #'', element(atom.atomicnum),
        atom.coords[0], atom.coords[1], atom.coords[2])
    
draw_html(pymol)

[*]C(=O)CCCCCCCCC1=C(C)C=C(CCCCC)O1
0, Du (0.969539414765, -0.0360597494272, 0.0881580808457)
6, C2 (1.96953941477, -0.0360597494272, 0.0881580808457)
8, O2 (2.60498941477, -0.648210022936, 1.00285046091)
6, C3 (2.72953941477, 0.696073711689, -1.0058165967)
6, C3 (2.36233554809, 0.107974345711, -2.35848080338)
6, C3 (0.867928659575, 0.262457039334, -2.58931223305)
6, C3 (0.500725637732, 1.73717256005, -2.56145413359)
6, C3 (1.26072563773, 2.46930602117, -3.65542881114)
6, C3 (0.893521771057, 1.88120665519, -5.00809301782)
6, C3 (-0.60088511746, 2.03568934881, -5.23892444749)
6, C3 (-0.968088139302, 3.51040486953, -5.21106634803)
6, Car (-0.208088139302, 4.24253833064, -6.30504102557)
6, Car (-0.31612624013, 5.52921063139, -6.49474911521)
6, C3 (-1.06873275444, 6.50958032401, -5.60995608834)
6, Car (0.366521071167, 5.82994743878, -7.62618847331)
6, Car (0.881617336122, 4.65875188744, -8.07291637524)
6, C3 (1.73755635639, 4.46702178619, -9.31428979281)
6, C3 (3.08737137995, 3.89322309839

In [4]:
for idx, row in molecules.iterrows():
    uff = OBUFFOptimize(smiles_string=row['original_smiles'])
    pm3 = OrcaOptimize()
    fw = Firework([uff, pm3])

    lpad.add_wf(fw);
 
        
def run(i):
    launch_rocket(lpad, FWorker())
    
run(1)

2018-03-13 12:42:03,549 INFO Added a workflow. id_map: {-1: 1}
2018-03-13 12:42:03,552 INFO Added a workflow. id_map: {-2: 2}
2018-03-13 12:42:03,556 INFO Added a workflow. id_map: {-3: 3}
2018-03-13 12:42:03,559 INFO Added a workflow. id_map: {-4: 4}
2018-03-13 12:42:03,563 INFO Added a workflow. id_map: {-5: 5}
2018-03-13 12:42:03,566 INFO Added a workflow. id_map: {-6: 6}
2018-03-13 12:42:03,570 INFO Added a workflow. id_map: {-7: 7}
2018-03-13 12:42:03,574 INFO Added a workflow. id_map: {-8: 8}
2018-03-13 12:42:03,579 INFO Added a workflow. id_map: {-9: 9}
2018-03-13 12:42:03,583 INFO Added a workflow. id_map: {-10: 10}
2018-03-13 12:42:03,586 INFO Added a workflow. id_map: {-11: 11}
2018-03-13 12:42:03,590 INFO Added a workflow. id_map: {-12: 12}
2018-03-13 12:42:03,593 INFO Added a workflow. id_map: {-13: 13}
2018-03-13 12:42:03,597 INFO Added a workflow. id_map: {-14: 14}
2018-03-13 12:42:03,600 INFO Added a workflow. id_map: {-15: 15}
2018-03-13 12:42:03,604 INFO Added a workfl

In [24]:
run(2)

2018-03-13 12:46:08,117 INFO Launching Rocket
2018-03-13 12:46:08,140 INFO RUNNING fw_id: 2 in directory: /home/bkrull/devel/MetAtlas
2018-03-13 12:46:08,146 INFO Task started: OBUFFOptimize.
2018-03-13 12:46:09,491 INFO Task completed: OBUFFOptimize 
2018-03-13 12:46:09,493 INFO Task started: OrcaOptimize.
2018-03-13 12:57:43,733 INFO Task completed: OrcaOptimize 
2018-03-13 12:57:45,999 INFO Rocket finished


## RDKit and/or OpenBabel do suspicious things

For some SMILES strings (luckily for me, the very first in the CSV), RDKit has a difficult time generating 3D conformers which are necessary for beginning our optimization process.

I noticed this because as I tried to automate making 3D coordinates for optimization, the first row in the CSV took an extraordinarily long time attempting to generate a conformer. I started to investigate and saw that it actually doesn't return a 3D conformer at all, which is very much an issue.

In [ ]:
ogsmile = molecules.loc[0, 'original_smiles']
sanismile = molecules.loc[0, 'sanitized_smiles']

rdkit_ogsmile = Chem.MolFromSmiles(ogsmile)
display(rdkit_ogsmile)
rdkit_ogsmile = Chem.AddHs(rdkit_ogsmile)
display(rdkit_ogsmile)

rdkit_sanismile = Chem.MolFromSmiles(sanismile)
display(rdkit_sanismile)
rdkit_sanismile = Chem.AddHs(rdkit_sanismile)
display(rdkit_sanismile)

Chem.MolToMolFile(rdkit_sanismile, 'mol0.mol')

In [ ]:
draw_html(pybel.readfile('mol', 'mol0.mol').next())

In [ ]:
idx = 19
print "Molecule: {}".format(molecules.loc[idx, 'name'])
ogsmile = molecules.loc[idx, 'original_smiles']
sanismile = molecules.loc[idx, 'sanitized_smiles']

rdkit_ogsmile = Chem.MolFromSmiles(ogsmile)
display(rdkit_ogsmile)
rdkit_ogsmile = Chem.AddHs(rdkit_ogsmile)
display(rdkit_ogsmile)

rdkit_sanismile = Chem.MolFromSmiles(sanismile)
display(rdkit_sanismile)
rdkit_sanismile = Chem.AddHs(rdkit_sanismile)
display(rdkit_sanismile)

Chem.MolToMolFile(rdkit_sanismile, 'mol19.mol')

In [ ]:
draw_html(pybel.readfile('mol', 'mol19.mol').next())

### An example of a SMILES -> rdkit.molecule that has an acceptable 3d conformer generated

In [ ]:
idx = 1
ogsmile = molecules.loc[idx, 'original_smiles']
sanismile = molecules.loc[idx, 'sanitized_smiles']

rdkit_ogsmile = Chem.MolFromSmiles(ogsmile)
display(rdkit_ogsmile)
rdkit_ogsmile = Chem.AddHs(rdkit_ogsmile)
display(rdkit_ogsmile)

rdkit_sanismile = Chem.MolFromSmiles(sanismile)
display(rdkit_sanismile)
rdkit_sanismile = Chem.AddHs(rdkit_sanismile)
display(rdkit_sanismile)

Chem.MolToMolFile(rdkit_sanismile, 'mol{}.mol'.format(idx))

In [ ]:
draw_html(pybel.readfile('mol', 'mol1.mol').next())

## OpenBabel is a little more sane

In [ ]:
pymol2d = pybel.readstring('smi', molecules.loc[0, 'original_smiles'])
pymol2d.removeh()
display(draw_html(pymol2d))

pymol2d.addh()
pymol2d.make3D()
# pymol.write('mol', filename='pymol3d.mol', overwrite=True)
display(draw_html(pymol2d))

In [ ]:
mol = molecules.loc[0, 'molecule']
m2 = Chem.AddHs(mol)

AllChem.EmbedMolecule(m2)
Chem.MolToMolFile(m2, 'mol.mol')

conf = m2.GetConformer()

In [ ]:
for i in tqdm(range(0,23)):
    mol = molecules.loc[i, 'molecule']
    m2 = Chem.AddHs(mol)
    if AllChem.EmbedMolecule(m2) < 0:
        print 'oh no {}'.format(i)

In [ ]:
Chem.rdMolDescriptors.CalcMolFormula(mol)

In [ ]:
mol = molecules.loc[2342, 'molecule']
m2 = Chem.AddHs(mol)

AllChem.EmbedMolecule(m2)
Chem.MolToMolFile(m2, 'mol.mol')

conf = m2.GetConformer()

natoms = m2.GetNumAtoms()
for idx in range(natoms):
    atom = m2.GetAtomWithIdx(idx)
    xyz = conf.GetAtomPosition(idx)
    print 'Index={}, Symbol={}, x={}, y={}, z={}'.format(atom_index, atom.GetSymbol(), xyz.x, xyz.y, xyz.z)

In [ ]:
""" crazy pybel/rdkit hybrid """
psi4.set_num_threads(8)
psi4.set_variable('print', 5)

""" Likely main.py structure """
for index, row in tqdm(molecules.iterrows()):
    smi = row['original_smiles']
    
    mol = Chem.MolFromSmiles(smi)
    mol = pybel.readstring('smi', smi)
    mol.removeh()

    mol.addh()
    mol.make3D()
    row['molecule'] = mol
    
    fname_mol = row['formula']+'.mol'
    fname_xyz = row['formula']+'.xyz'
    mol.write('mol', fname_mol, overwrite=True)

    mol = Chem.MolFromMolFile(fname_mol)
    mol = AllChem.AddHs(mol)
    try:
        AllChem.EmbedMolecule(mol)    
        AllChem.UFFOptimizeMolecule(mol)
        display(mol)
    except:
        continue
    
    Chem.MolToMolFile(mol, fname_mol)
    
    mol = pybel.readfile('mol', fname_mol).next()
    mol.write('xyz', fname_xyz, overwrite=True)
    mol = xyzfile_to_psi4mol(fname_xyz)

    e, wfn = psi4.optimize('pbe/sto-3g', molecule=mol, return_wfn='on')
    if index > 5: break

In [ ]:
""" crazy rdkit only """
psi4.set_num_threads(8)
psi4.set_variable('print', 5)

""" Likely main.py structure """
for index, row in tqdm(molecules.iterrows()):
    smi = row['original_smiles']
    
    mol = Chem.MolFromSmiles(smi)
    mol = AllChem.AddHs(mol)
    row['molecule'] = mol

    try:
        AllChem.EmbedMolecule(mol)    
        AllChem.UFFOptimizeMolecule(mol)
        display(mol)
    except:
        continue
    
    fname_mol = row['formula']+'.mol'
    fname_xyz = row['formula']+'.xyz'
    
    Chem.MolToMolFile(mol, fname_mol)
    
    mol = pybel.readfile('mol', fname_mol).next()
    
    if len(mol.atoms) > 20: continue
    mol.write('xyz', fname_xyz, overwrite=True)
    mol = xyzfile_to_psi4mol(fname_xyz)

    e, wfn = psi4.optimize('pbe/sto-3g', molecule=mol, return_wfn='on')
    if index > 5: break
    #psi4.optimize('pbeh3c/def-svp', molecule=mol)

## Optimizing structures and getting output from psi4

In [ ]:

mol = xyzfile_to_psi4mol('C3H10NO2+.xyz')

e, wfn = psi4.optimize('pbe/sto-3g', molecule=mol, return_wfn='on')

In [ ]:
np.asarray(wfn.Fa())